In [57]:
import string
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import regularizers

In [21]:
database = pd.read_csv("rsos_dream_data.csv")
df = database[["text_dream", "Male", "Animal", "Friends", "Family", "Dead&Imaginary",
                            "Aggression/Friendliness", "A/CIndex", "F/CIndex", "S/CIndex", "NegativeEmotions"]]

df['text_dream'] = df['text_dream'].apply(lambda x: x.lower())
df['text_dream'] = df['text_dream'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['text_dream'] = df['text_dream'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))
df['text_dream'] = df['text_dream'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Vectorize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text_dream'])
sequences = tokenizer.texts_to_sequences(df['text_dream'])
X = pad_sequences(sequences, maxlen=100)



C:\Users\Haochen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Haochen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Haochen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, df[
    ['Male', 'Animal', 'Friends', 'Family', 'Dead&Imaginary', 'Aggression/Friendliness', 'A/CIndex', 'F/CIndex',
     'S/CIndex', 'NegativeEmotions']], test_size=0.2)

model = Sequential()
model.add(Embedding(5000, 128, input_length=100))
model.add(LSTM(128, kernel_regularizer=regularizers.L1(0.01)))
model.add(Dense(10, activation='softmax', kernel_regularizer=regularizers.L1(0.01)))

model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(X_train, y_train, epochs=30, verbose=1, batch_size= 128, validation_data=(X_test, y_test),
          callbacks=[EarlyStopping(monitor='val_loss', patience=20)])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.3}')
print(f'Test mse: {accuracy:.3}')

prediction = model.predict(X_test)
print(prediction)

Epoch 1/30
132/132 [==============================] - 14s 91ms/step - loss: 8.9882 - mse: 0.1699 - val_loss: 0.4699 - val_mse: 0.1753

In [35]:

# Preprocess the text
text = df['text_dream'][15]
print(df.iloc[15])
text = tokenizer.texts_to_sequences([text])
text = pad_sequences(text, maxlen=100)

# Predict the scores for each category
scores = model.predict(text)
print(scores)



me pulling the green leaves and berries off some branches or a bush not from a live one growing in the ground but inside the house a boy of  or  is impatiently telling me mothers asleep i think  as if im not supposed to be doing what im doing instead of her or without her knowledge or something this is in an old rambling house with lots of old furniture and interconnected rooms lace doilies on little tables throw rugs etc some of us family friends  theres  or  of us i think  are outside and theres a dog on the rampage  didnt we think it was a wolf at first anyway it turns out to be a black boxer or bulldog or something and it charges the back door theres some kind of flap or opening in it that i plug up  i dont know what the people outside are doing but theyre okay and they come in shortly somebody has rather by accident invited somebody like a neighbor to come over for something on the order of borrowing a cup of sugar trouble is this person always has a party with himher kind of a gr

In [62]:
# Save the entire model
model.save('model.h5')

In [ ]:
from keras.models import load_model
# Load the entire model
model = load_model('model.h5')
